Assuming you have a dataframe containing video data with details such as the number of comments, the number of hate comments, the description, and the video cluster, this code calculates the results outlined in Section 6 on Hate Speech Regularities.

In [ ]:
# Standard Library Imports
import json
import ast

# Third-Party Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import load
import shap

# Scikit-Learn Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_squared_error, 
    accuracy_score
)

# SHAP Values computation
In this implementation, one can train and finetune both the logistic and linear regressions as desired.

In [ ]:
# Prepare data for logistic regression
videoData = videoDataWithOutliers[videoDataWithOutliers['bertTopicVideoFrameAndDescriptionCluster'] != -1].copy()
videoData['has_hate_comments'] = videoData['hate_comments'] > 0

# Define features and target variable
distance_columns = [col for col in videoData.columns if 'Topic_' in col and '_distance' in col] 
X = videoData[distance_columns]
y = videoData['has_hate_comments']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features as desired
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the logistic regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train_scaled, y_train)

# Predict using logistic regression
y_pred_logistic = logistic_model.predict(X_test_scaled)

# Calculate accuracy for logistic regression
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)

# SHAP values for logistic regression
explainer_logistic = shap.LinearExplainer(logistic_model, X_train_scaled, feature_perturbation="correlation_dependent")
shap_values_logistic = explainer_logistic.shap_values(X_test_scaled)

# Calculate mean absolute SHAP values for logistic regression
shap_values_logistic_mean_abs = pd.DataFrame(shap_values_logistic, columns=distance_columns).mean()
top_logistic_features = shap_values_logistic_mean_abs.sort_values(ascending=False).head(10)

# Extract numeric identifiers from the 'Feature' column for merging
top_logistic_features_df = top_logistic_features.reset_index().rename(columns={0: 'MeanSHAP', 'index': 'Feature'})
top_logistic_features_df['Topic_num'] = top_logistic_features_df['Feature'].str.extract('(\d+)').astype(int)

# Merge top features with their representations
unique_representations = videoData[['bertTopicVideoFrameAndDescriptionCluster', 'Representation']].drop_duplicates(subset=['bertTopicVideoFrameAndDescriptionCluster'])
unique_representations['bertTopicVideoFrameAndDescriptionCluster'] = unique_representations['bertTopicVideoFrameAndDescriptionCluster'].astype(int)
top_10_logistic = top_logistic_features_df.merge(unique_representations, left_on='Topic_num', right_on='bertTopicVideoFrameAndDescriptionCluster', how='left')

# Calculate overall average SHAP values for all features
overall_avg_shap_logistic = pd.DataFrame(shap_values_logistic, columns=distance_columns).mean().mean()
top_10_logistic['OverallAvgSHAP'] = overall_avg_shap_logistic
top_10_logistic['SHAP_vs_Avg'] = top_10_logistic['MeanSHAP'] / top_10_logistic['OverallAvgSHAP']

# Print results for Logistic Regression
print('Top 10 Most Influential Features from Logistic Regression (SHAP values):')
print(top_10_logistic[['Feature', 'MeanSHAP', 'OverallAvgSHAP', 'SHAP_vs_Avg', 'Representation']])
print("Logistic Regression Accuracy:", accuracy_logistic)


In [ ]:
# Prepare data for linear regression
videoData = videoDataWithOutliers[videoDataWithOutliers['bertTopicVideoFrameAndDescriptionCluster'] != -1].copy()
videoData['hate_comment_percentage'] = videoData['hate_comments'] / videoData['total_comments']
videoData['hate_comment_percentage'] = videoData['hate_comment_percentage'].fillna(0)  # Fill NaN values

# Define features and target variable
distance_columns = [col for col in videoData.columns if 'Topic_' in col and '_distance' in col] 
X = videoData[distance_columns]
y_lr = videoData['hate_comment_percentage']

# Split the data into training and testing sets
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X, y_lr, test_size=0.2, random_state=42)

# Standardize the features as desired
scaler_lr = StandardScaler()
X_train_lr_scaled = scaler_lr.fit_transform(X_train_lr)
X_test_lr_scaled = scaler_lr.transform(X_test_lr)

# Train the linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train_lr_scaled, y_train_lr)

# Predict with linear regression and evaluate using regression metrics
y_pred_lr = linear_model.predict(X_test_lr_scaled)
mse_lr = mean_squared_error(y_test_lr, y_pred_lr)

# SHAP values for linear regression
explainer_lr = shap.LinearExplainer(linear_model, X_train_lr_scaled, feature_perturbation="correlation_dependent")
shap_values_lr = explainer_lr.shap_values(X_test_lr_scaled)

# Calculate mean absolute SHAP values for linear regression
shap_values_lr_mean_abs = pd.DataFrame(shap_values_lr, columns=distance_columns).mean()
top_lr_features = shap_values_lr_mean_abs.sort_values(ascending=False).head(10)

# Extract numeric identifiers from the 'Feature' column for merging
top_lr_features_df = top_lr_features.reset_index().rename(columns={0: 'MeanSHAP', 'index': 'Feature'})
top_lr_features_df['Topic_num'] = top_lr_features_df['Feature'].str.extract('(\d+)').astype(int)

# Merge top features with their representations
unique_representations = videoData[['bertTopicVideoFrameAndDescriptionCluster', 'Representation']].drop_duplicates(subset=['bertTopicVideoFrameAndDescriptionCluster'])
unique_representations['bertTopicVideoFrameAndDescriptionCluster'] = unique_representations['bertTopicVideoFrameAndDescriptionCluster'].astype(int)
top_10_lr = top_lr_features_df.merge(unique_representations, left_on='Topic_num', right_on='bertTopicVideoFrameAndDescriptionCluster', how='left')

# Calculate overall average SHAP values for all features
overall_avg_shap_lr = pd.DataFrame(shap_values_lr, columns=distance_columns).mean().mean()
top_10_lr['OverallAvgSHAP'] = overall_avg_shap_lr
top_10_lr['SHAP_vs_Avg'] = top_10_lr['MeanSHAP'] / top_10_lr['OverallAvgSHAP']

# Print results for Linear Regression
print('Top 10 Most Influential Features from Linear Regression (SHAP values):')
print(top_10_lr[['Feature', 'MeanSHAP', 'OverallAvgSHAP', 'SHAP_vs_Avg', 'Representation']])
print("Linear Regression MSE:", mse_lr)


# Top Clusters by hate

In [ ]:
# Filter out rows where 'bertTopicVideoFrameAndDescriptionCluster' is -1
videoData = videoDataWithOutliers[videoDataWithOutliers['bertTopicVideoFrameAndDescriptionCluster'] != -1]
videoData['hate_comment_percentage'] = videoData['hate_comment_percentage'].fillna(0)
cluster_hate_percentage = videoData.groupby('bertTopicVideoFrameAndDescriptionCluster')['hate_comment_percentage'].mean().reset_index()

cluster_info = videoData[['bertTopicVideoFrameAndDescriptionCluster', 'Count', 'Representation']].drop_duplicates()
cluster_hate_percentage = cluster_hate_percentage.merge(cluster_info, on='bertTopicVideoFrameAndDescriptionCluster', how='left')
sorted_clusters = cluster_hate_percentage.sort_values(by='hate_comment_percentage', ascending=False)

# Display the top 50 clusters ranked by hate comment percentage
print('Top 50 Clusters ranked by hate comment percentage:')
top_50_clusters_percentage = sorted_clusters.head(50)

# Also sort the clusters by 'Count' in descending order
sorted_clusters_by_count = cluster_hate_percentage.sort_values(by='Count', ascending=False)
top_50_clusters_count = sorted_clusters_by_count.head(50)

# Print the top clusters by percentage
print("Top 50 Clusters by Hate Comment Percentage:")
print(top_50_clusters_percentage[['bertTopicVideoFrameAndDescriptionCluster', 'hate_comment_percentage', 'Count', 'Representation']])

# Hashtaghs

In [ ]:
# Function to extract unique pairs of hashtags from a text
def extract_hashtag_pairs(text):
    hashtags = re.findall(r'#\w+', text)
    # Use itertools.combinations to find all pairs, if no pairs, just the hashtag itself
    if len(hashtags) < 2:
        return hashtags
    else:
        pairs = list(itertools.combinations(sorted(set(hashtags)), 2))
        return pairs + hashtags  # include individual hashtags as well

def process_hashtag_pairs(hashtags_df):
    # Group by hashtag pairs and compute required metrics
    hashtag_metrics = hashtags_df.groupby('hashtag_pairs').agg(
        hashtag_count=('hashtag_pairs', 'size'),
        total_comments=('total_comments', 'sum'),
        total_hate_comments=('hate_comments', 'sum'),
        total_hate_percentage=('hate_comment_percentage', 'sum'),
    ).reset_index()

    if 'aweme_id' in hashtags_df.columns:
        n_videos = len(hashtags_df['aweme_id'].unique())
    else:
        n_videos = len(hashtags_df)

    total_comments = np.sum(hashtags_df.drop_duplicates(subset=["aweme_id"])["total_comments"])

    hashtag_metrics['average_hate_percentage'] = hashtag_metrics['total_hate_percentage'] / hashtag_metrics['hashtag_count']
    
    ranked_by_hate_comments = hashtag_metrics.sort_values(by='hashtag_count', ascending=False).head(400)
    ranked_by_hate_comments = ranked_by_hate_comments.sort_values(by='total_hate_comments', ascending=False).head(50)
    top_hate_percentage = ranked_by_hate_comments.sort_values(by='average_hate_percentage', ascending=False).head(30)

    return top_hate_percentage


In [ ]:
videoData.loc[:, 'desc'] = videoData['desc'].fillna('')

videoData.loc[:, 'hashtag_pairs'] = videoData['desc'].apply(extract_hashtag_pairs)
hashtags_df = videoData.explode('hashtag_pairs')

# Process the data
hashtag_metrics = process_hashtag_pairs(hashtags_df)
hashtag_metrics